# Building Supervised AI Model to forecast sales anomaly
Method: TF Neural Network Classifier

Process:

Modelling data by aggregate orders line and merging it to orders info
Predict standard value of HMS product within orders
Differrence between predicted value and actual value will define anomaly flag
Features to be used:

- Wholesaler
- Retailer
- Line Count
- Line of HMS Product
- Sum Qty of HMS
- Value HMS

Supervised label: anomaly_label

In [42]:
import numpy as np
import pandas as pd
import tensorflow as tf

from sklearn.model_selection import train_test_split
from tensorflow.keras import layers
from tensorflow.keras.layers.experimental import preprocessing

## Read Data, check for missing field and apply defaut value

In [43]:
orders = pd.read_csv("datasource/train_data/train_orders.csv")
prods = pd.read_csv("datasource/train_data/train_order_products.csv")
print("orders:",len(orders))
print("prods:",len(prods))

orders: 10000
prods: 113606


In [44]:
orders.isna().sum()
orders.head(2)

,id,wholesaler_id,retailer_id,buyer_type,shipping_type,order_type,book_time,last_status,last_status_time,anomaly_label
0,5270143,154524,104735,retailer,pick-up,app,8/29/2020 15:53,siap-diambil,8/29/2020 16:06,0
1,5331813,137063,103017,retailer,pick-up,app,9/1/2020 16:10,siap-diambil,9/8/2020 11:24,0


In [45]:
prods["brand"].fillna('Other', inplace=True)
prods["packaging_amount"].fillna(1, inplace=True)
prods.isna().sum()

id                  0
sku_id              0
brand               0
packaging           0
packaging_amount    0
amount              0
price               0
book_time           0
last_status         0
last_status_time    0
dtype: int64

## Create new features

In [46]:
prods["is_hms"] = prods.apply(lambda x: 1 if str(x.brand).lower().find("sampoerna") > 0 else 0, axis=1)
prods["qty"] = prods["packaging_amount"] * prods["amount"]
prods["value"] = prods["price"] * prods["amount"]
prods["qty_hms"] = prods.apply(lambda x: x.qty if x.is_hms==1 else 0, axis=1)
prods["value_hms"] = prods.apply(lambda x: x.value if x.is_hms==1 else 0, axis=1)

In [47]:
orders["trx_time"] = pd.to_datetime(orders["book_time"])
orders["trx_date"] = orders["trx_time"].dt.year*10000+orders["trx_time"].dt.month*100+orders["trx_time"].dt.day
orders.head(2)

,id,wholesaler_id,retailer_id,buyer_type,shipping_type,order_type,book_time,last_status,last_status_time,anomaly_label,trx_time,trx_date
0,5270143,154524,104735,retailer,pick-up,app,8/29/2020 15:53,siap-diambil,8/29/2020 16:06,0,2020-08-29 15:53:00,20200829
1,5331813,137063,103017,retailer,pick-up,app,9/1/2020 16:10,siap-diambil,9/8/2020 11:24,0,2020-09-01 16:10:00,20200901


## Aggregate order line and join it to orders

In [48]:
prod_sum = prods.groupby("id").agg(
    {"is_hms":{"line_count":"count","line_hms":"sum"}, 
    "qty":{"qty":"sum"}, "qty_hms":{"qty_hms":"sum"},"value":{"value":"sum"}, "value_hms":{"value_hms":"sum"}})
prod_sum.columns = prod_sum.columns.droplevel(0)
prod_sum = prod_sum.reset_index()

orders_sum = orders.merge(prod_sum, how="left", on="id")
orders_sum["ts"] = orders_sum.apply(lambda x: x["trx_time"].timestamp(), axis=1)
orders_sum.head(2)

,id,wholesaler_id,retailer_id,buyer_type,shipping_type,order_type,book_time,last_status,last_status_time,anomaly_label,trx_time,trx_date,line_count,line_hms,qty,qty_hms,value,value_hms,ts
0,5270143,154524,104735,retailer,pick-up,app,8/29/2020 15:53,siap-diambil,8/29/2020 16:06,0,2020-08-29 15:53:00,20200829,6,6,44.0,44.0,844750,844750,1.598716e+09
1,5331813,137063,103017,retailer,pick-up,app,9/1/2020 16:10,siap-diambil,9/8/2020 11:24,0,2020-09-01 16:10:00,20200901,1,1,20.0,20.0,316000,316000,1.598977e+09


## AI Model Functions

In [49]:
# A utility method to create a tf.data dataset from a Pandas Dataframe
def df_to_dataset(dataframe, target_name, shuffle=True, batch_size=32):
  dataframe = dataframe.copy().rename(columns={target_name:'target'})
  labels = dataframe.pop('target')
  ds = tf.data.Dataset.from_tensor_slices((dict(dataframe), labels))
  if shuffle:
    ds = ds.shuffle(buffer_size=len(dataframe))
  ds = ds.batch(batch_size)
  ds = ds.prefetch(batch_size)
  return ds

def get_normalization_layer(name, dataset):
  # Create a Normalization layer for our feature.
  normalizer = preprocessing.Normalization()

  # Prepare a Dataset that only yields our feature.
  feature_ds = dataset.map(lambda x, y: x[name])

  # Learn the statistics of the data.
  normalizer.adapt(feature_ds)

  return normalizer

def get_category_encoding_layer(name, dataset, dtype, max_tokens=None):
  # Create a StringLookup layer which will turn strings into integer indices
  if dtype == 'string':
    index = preprocessing.StringLookup(max_tokens=max_tokens)
  else:
    index = preprocessing.IntegerLookup(max_values=max_tokens)

  # Prepare a Dataset that only yields our feature
  feature_ds = dataset.map(lambda x, y: x[name])

  # Learn the set of possible values and assign them a fixed integer index.
  index.adapt(feature_ds)

  # Create a Discretization for our integer indices.
  encoder = preprocessing.CategoryEncoding(max_tokens=index.vocab_size())

  # Prepare a Dataset that only yields our feature.
  feature_ds = feature_ds.map(index)

  # Learn the space of possible indices.
  encoder.adapt(feature_ds)

  # Apply one-hot encoding to our indices. The lambda function captures the
  # layer so we can use them, or include them in the functional model later.
  return lambda feature: encoder(index(feature))

In [54]:
def make_aimodel(model_name='model', wsid=[], test_size=0.2, epochs=10, batch_size=256):
    if len(wsid) <= 5:
        print("Build AI model for wholesaler {0}".format(wsid))
    else:
        print("Build AI model for wholesalers {0}...".format(wsid[:4]))

    cols = ["wholesaler_id","retailer_id", "line_count","line_hms","value","value_hms","qty_hms","anomaly_label"]
    data = orders_sum.drop(columns=["id","trx_date"]) if len(wsid)==0 else orders_sum[orders_sum["wholesaler_id"].isin(wsid)].drop(columns=["id","trx_date"])
    data = data[cols]
    print("Data count: {0}".format(len(data)))
    
    train, val = train_test_split(data, test_size=test_size)
    train, test = train_test_split(train, test_size=test_size)

    train_ds = df_to_dataset(train, batch_size=batch_size, target_name="anomaly_label")
    val_ds = df_to_dataset(val, shuffle=False, batch_size=batch_size, target_name="anomaly_label")
    test_ds = df_to_dataset(test, shuffle=False, batch_size=batch_size, target_name="anomaly_label")

    category_int_columns = ["wholesaler_id","retailer_id"]
    category_str_columns = []
    numerical_columns = ["line_count","line_hms","value","value_hms","qty_hms"]
    inputs = []
    features = []

    # numerical cols
    for header in numerical_columns:
        numeric_col = tf.keras.Input(shape=(1,), name=header)
        normalization_layer = get_normalization_layer(header, train_ds)
        encoded_numeric_col = normalization_layer(numeric_col)
        inputs.append(numeric_col)
        features.append(encoded_numeric_col)

    # categorical cols (int)
    for header in category_int_columns:
        categorical_col = tf.keras.Input(shape=(1,), name=header, dtype='int64')
        encoding_layer = get_category_encoding_layer(header, train_ds, dtype='int64')
        encoded_categorical_col = encoding_layer(categorical_col)
        inputs.append(categorical_col)
        features.append(encoded_categorical_col)

    # categorical cols (str)
    for header in category_str_columns:
        categorical_col = tf.keras.Input(shape=(1,), name=header, dtype='string')
        encoding_layer = get_category_encoding_layer(header, train_ds, dtype='string')
        encoded_categorical_col = encoding_layer(categorical_col)
        inputs.append(categorical_col)
        features.append(encoded_categorical_col)

    # the model
    model_features = tf.keras.layers.concatenate(features)
    x = tf.keras.layers.Dense(32, activation="relu")(model_features)
    x = tf.keras.layers.Dropout(0.5)(x)
    output = tf.keras.layers.Dense(1)(x)
    model = tf.keras.Model(inputs, output)
    model.compile(optimizer='adam',
                loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
                metrics=["accuracy"])

    # visualize the model
    # tf.keras.utils.plot_model(model, show_shapes=True, rankdir="LR")

    # train
    model.fit(train_ds, epochs=epochs, validation_data=test_ds)
    model.save("aimodel/{0}".format(model_name))

    # check accuracy
    loss, accuracy = model.evaluate(val_ds)
    print("Accuracy", accuracy)
    print("")

## Create the model and save

In [55]:
make_aimodel("ws_all", epochs=100)

Build AI model for wholesaler []
Data count: 10000
Epoch 1/100
25/25 [==============================] - 0s 17ms/step - loss: 0.6533 - accuracy: 0.9609 - val_loss: 0.6044 - val_accuracy: 0.9669
Epoch 2/100
25/25 [==============================] - 0s 8ms/step - loss: 0.5629 - accuracy: 0.9613 - val_loss: 0.5140 - val_accuracy: 0.9669
Epoch 3/100
25/25 [==============================] - 0s 9ms/step - loss: 0.4692 - accuracy: 0.9609 - val_loss: 0.4233 - val_accuracy: 0.9669
Epoch 4/100
25/25 [==============================] - 0s 9ms/step - loss: 0.3818 - accuracy: 0.9609 - val_loss: 0.3413 - val_accuracy: 0.9669
Epoch 5/100
25/25 [==============================] - 0s 8ms/step - loss: 0.3057 - accuracy: 0.9614 - val_loss: 0.2736 - val_accuracy: 0.9669
Epoch 6/100
25/25 [==============================] - 0s 9ms/step - loss: 0.2477 - accuracy: 0.9634 - val_loss: 0.2227 - val_accuracy: 0.9669
Epoch 7/100
25/25 [==============================] - 0s 8ms/step - loss: 0.2048 - accuracy: 0.9683 - v